In [1]:
import uuid

import babel

from seis_lab_data import (
    config,
    operations,
    schemas,
)
from seis_lab_data.db.engine import (
    get_engine,
    get_session_maker,
)

EN_LOCALE = babel.Locale.parse("en_US")
PT_LOCALE = babel.Locale.parse("pt_PT")

settings = config.get_settings()

SyntaxError: expected ':' (marinecampaigns.py, line 45)

In [2]:
to_create = schemas.MarineCampaignCreate(
    id=uuid.uuid4(),
    owner="some-guy",
    name={
        EN_LOCALE.language: "my beautiful campaign",
        PT_LOCALE.language: "A minha belíssima campanha",
    },
    links=[
        {
            "url": "http://fake.com",
            "media_type": "text/html",
            "relation": "parent",
            "description": {EN_LOCALE.language: "This is a link about something"},
        }
    ],
)

In [ ]:
engine = get_engine(settings)
session_maker = get_session_maker(engine)

In [3]:
async with session_maker() as session:
    c1 = await operations.create_marine_campaign(to_create, settings)

2025-08-27 16:49:10,594 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-08-27 16:49:10,594 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-27 16:49:10,596 INFO sqlalchemy.engine.Engine select current_schema()
2025-08-27 16:49:10,596 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-27 16:49:10,597 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-08-27 16:49:10,598 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-27 16:49:10,602 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-27 16:49:10,603 INFO sqlalchemy.engine.Engine INSERT INTO marinecampaign (id, owner, name, slug, links) VALUES (%(id)s::UUID, %(owner)s::VARCHAR, %(name)s::JSONB, %(slug)s::VARCHAR, %(links)s::JSONB)
2025-08-27 16:49:10,604 INFO sqlalchemy.engine.Engine [generated in 0.00053s] {'id': UUID('232c2349-6adb-4b79-bcea-88732b514d07'), 'owner': 'some-guy', 'name': Jsonb({'en': 'my beautiful campaign', 'pt ... (67 chars)), 'slug': 'my-beautiful-campaign', 'links': Jsonb([{'url